# Quantum simulation of the 2D Heisenberg Model

In this Python + Q# notebook we demonstrate resource estimation for simulating a Heisenberg model Hamiltonian on an $N \times N$ 2D
lattice using a *fourth-order Trotter Suzuki product formula* assuming a 2D planar qubit architecture with nearest-neighbor connectivity.

First, we connect to the Azure quantum service and load the necessary packages.

In [ ]:
from azure.quantum import Workspace
from azure.quantum.target.microsoft import MicrosoftEstimator, QubitParams, QECScheme
import qsharp


In [ ]:
workspace = Workspace (
   resource_id = "",
   location = ""
)

In [ ]:
qsharp.packages.add("Microsoft.Quantum.Numerics")

## Background: 2D Heisenberg Model

The quantum Heisenberg model is a statistical mechanical model used in the study of magnetic systems. The family of Heisenberg model Hamiltonians considered in this notebook consist of three types of interaction terms between adjacent lattice sites: $\{X \otimes X, Y \otimes Y, Z \otimes Z\}$. For our purposes we consider that the interaction strength $J$ is the same for each term. Formally, the Heisenberg model Hamiltonian on an $N \times N$ lattices divided into sets of commuting terms is formulated as:
$$ H = \underbrace{J \sum_{i, j} X_i \otimes X_j}_{A} + \underbrace{J \sum_{i, j} Z_i \otimes Z_j}_{B} + \underbrace{J \sum_{i, j} Y_i \otimes Y_j}_{C}.$$

TThe time evolution $e^{-iHt}$ for the Hamiltonian is simulated with the fourth-order Trotter-Suzuki product formula so that any errors in simulation are sufficiently small. Essentially, this is done by simulating the evolution for small slices of time $\Delta$ and repeating this for `nSteps` $= \lceil t/\Delta \rceil$ to obtain the full time evolution. The Trotter-Suzuki formula for higher orders can be recursively defined using a *fractal decomposition* as discussed in Section 3 of [Hatanao and Suziki's survey](https://link.springer.com/chapter/10.1007/11526216_2). Then the fourth order formula $U_4(\Delta)$ can be constructed using the second-order one $U_2(\Delta)$ as follows.
$$
\begin{aligned}
U_2(\Delta) & = e^{-iA\Delta/2} e^{-iB\Delta/2} e^{-iC\Delta} e^{-iB\Delta/2} e^{-iA\Delta/2}; \\
U_4(\Delta) & = U_2(p\Delta)U_2(p\Delta)U_2((1 - 4p)\Delta)U_2(p\Delta)U_2(p\Delta); \\
p & = (4 - 4^{1/3})^{-1}.
\end{aligned}
$$

For the rest of the notebook, we will present the code that computes the time evolution in a step by step fashion.

## Implementation

### Helper Functions

Note that expanding $U_4(\Delta)$ to express it in terms of $A, B, C$ gives:
$$
\begin{aligned}
U_4(\Delta) & = e^{-iAp\Delta/2} \ T_1 \ T_2 \ T_1 \ e^{-iAp\Delta/2}, \\
\text{where } T_1 & = e^{-iBp\Delta/2} e^{-iCp\Delta} e^{-iBp\Delta/2} e^{-iAp\Delta} e^{-iBp\Delta/2} e^{-iCp\Delta} e^{-iBp\Delta/2}, \\
\text{and } T_2 & = e^{-iA(1 - 3p)\Delta/2} e^{-iB(1-4p)\Delta/2} e^{-iC(1-4p)\Delta} e^{-iB(1-4p)\Delta/2} e^{-iA(1 - 3p)\Delta/2}.
\end{aligned}
$$

The above equation has $21$ exponential terms and works for one time step. For `nSteps` $> 1$ time steps, some adjacent terms can be merged to give $20\lceil t/\Delta \rceil+1$ exponential terms for time evolution $e^{-iHt}$.

The function below sets the rotation angles used to apply the exponentials involving $A, B$ and $C$ in the formula above.

In [ ]:
%%qsharp
function SetAngleSequence(p : Double, dt : Double, J : Double) : Double[] {

    let len1 = 8;
    let len2 = 4;
    let valLength = 2*len1 + len2;

    mutable values = [0.0, size=valLength];

    let val1 = -J*p*dt/2.0;
    let val2 = -J*(1.0-3.0*p)*dt/2.0;
    let val3 = -J*(1.0-4.0*p)*dt/2.0;

    // Angles bookending Term2
    set values w/= len1 <- val2;
    set values w/= len1+len2 <- val2;

    for i in 0..len1-1 {
        
        if (i % 2 == 0) {
            set values w/= i <- val1*2.0;
        }
        else {
            set values w/= i <- val1;
        }
    }

    for i in len1+1..len1+len2-1 {
        
        if (i % 2 == 0) {
            set values w/= i <- val3*2.0;
        }
        else {
            set values w/= i <- val3;
        }
    }

    for i in len1+len2+1..valLength-1 {
        
        if (i % 2 == 0) {
            set values w/= i <- val1*2.0;
        }
        else {
            set values w/= i <- val1;
        }
    }

    return values;
}

### Quantum operations

There are three kinds of operations applied to neighboring pairs of qubits on the lattice: (i) $e^{-i(X \otimes X)\theta}$; (ii) $e^{-i(Z \otimes Z)\theta}$; and (iii) $e^{-i(Y \otimes Y)\theta}$.

The following operation applies $e^{i(P \otimes P)\theta}$ for Pauli $P \in \{X, Y, Z\}$ on pairs of horizinontally (resp. vertically) neighboring qubits in a single row of the lattice if `dir` is `True` (resp. `False`). Additionally, the exponential is applied only to pairs where the first qubit id is even (resp. odd) if `grp` is `True` (resp. `False`).

In [ ]:
%%qsharp
operation ApplyPoPRot(n : Int, m : Int, qArr : Qubit[][], P : Pauli, theta : Double, dir : Bool, grp : Bool) : Unit {

    let start = (grp ? 0 | 1);    // Choose either odd or even indices based on group number
    let P_op = [P, P];
    let jmp = 2;
    let end = dir ? m-2 | m-1;
    let v_end = dir? n-1 | n-2;

    for row in 0..v_end {
        for col in start..jmp..end {    // Either odd or even columns are picked based on group number
            
            let row2 = dir? row | row+1;    // Choose the next row if for vertical direction 
            let col2 = dir? col+1 | col;    // Choose the next column for horizontal direction

            Exp(P_op, theta, [qArr[row][col], qArr[row2][col2]]);
        }
    }
}

The next operation puts everything together and calls the operations needed to
simulate the Heisenberg model Hamiltonian using a fourth order product formula.
Observe that the `ApplyPoPRot` operation is called four times for each row for different
choices of direction and starting index to ensure all possible pairs of qubits
are appropriately considered.

The various parameters taken in by the operation correspond to:

- `N1`, `N2`: row and columns size of the lattice.
- `J`: parameter by which the Hamiltonian terms are scaled.
- `totTime`: the total time for the Trotter simulation.
- `dt` : the step size for the simulation, sometimes denoted as $\Delta$.

In [ ]:
%%qsharp
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;

operation HeisModel2DSim(N1 : Int, N2 : Int, J : Double, totTime : Double, dt : Double) : Unit {

    use qs = Qubit[N1*N2];

    let qubitArray = Chunks(N2, qs); // Create a 2D array of qubits

    let len = Length(qs);

    let p = 1.0/(4.0 - PowD(4.0, 1.0/3.0));

    let t = Ceiling(totTime/dt);

    let seqLen = 20*t+1;

    let angSeq = SetAngleSequence(p, dt, J);

    let paulis = [PauliX, PauliZ, PauliY];

    for i in 0..seqLen-1 {
        
        let j = (i%4==0) ? 0 | ( (i%2==1) ? 1 | 2); // Choose which pauli rotation will be applied
        let theta = (i==0 or i==seqLen-1) ? -J*p*dt/2.0 | angSeq[i%20]; 

        for (dir, grp) in [(true, true), (true, false), (false, true), (false, false)] {
            ApplyPoPRot(N1, N2, qubitArray, paulis[j], theta, dir, grp);
        }
    }

    //Resetting qubits before release;
    ResetAll(qs);
}

## Getting logical resource counts

For the purpose of generating the rQOPS for some target runtime, it suffices that we obtain the logical resource estimates to simulate the Heisenberg model Hamiltonian. We consider three problem instances with lattice sizes $\{20 \times 20, 30 \times 30, 40 \times 40\}$ with $J = 1.0$. These instances are simulated for a total time of $1000$s, with step size `dt`$ = 0.1$, and overall probability of failure $\varepsilon = 0.01$. Any one of the six pre-defined qubit parameters will do to obtain the logical coounts and in this notebook we choose a Majorana based qubit with the `floquet code`.

In [ ]:
estimator = MicrosoftEstimator(workspace)

labels = ["Hei-20", "Hei-30", "Hei-40"]

params = estimator.make_params(num_items=3)
params.items[0].arguments["N1"] = 20
params.items[0].arguments["N2"] = 20
params.items[1].arguments["N1"] = 30
params.items[1].arguments["N2"] = 30
params.items[2].arguments["N1"] = 40
params.items[2].arguments["N2"] = 40
params.arguments["totTime"] = 1000.0
params.arguments["J"] = 1.0
params.arguments["g"] = 1.0
params.arguments["dt"] = 0.1
params.error_budget = 0.01
params.qubit_params.name = QubitParams.MAJ_NS_E4
params.qec_scheme.name = QECScheme.FLOQUET_CODE
params.constraints.logical_depth_factor=4

We submit a resource estimation job with all the problem instances simultaneously.

In [ ]:
job = estimator.submit(HeisModel2DSim, input_params=params)
results = job.get_results()

## Visualizing and understanding the results

### Result summary table

In [ ]:
results.summary_data_frame(labels=labels)

While the above table provides the logical qubits and depth that are needed to compute the target rQOPS, we use the full set of results to also get the required logical qubit error rate.

In [ ]:
results[0:3]

> Note that in general, there is a trade-off between the logical depth and number of T factories used. 

> To ensure that T factories do not dominate the resource requirements, we set the `logical_depth_factor`${}=4$ adding some number of `noops` to increase the logical depth.

## Next steps

Feel free to use this notebook as a starting point for your own experiments.  For
example, you can

* explore how the results change considering other problem instances of the Heisenberg model
* explore space- and time-trade-offs by changing the value for
  `logical_depth_factor` or `max_t_factories`
* visualize these trade-offs with the space and time diagrams
* use other or customized qubit parameters